**Exercise 3**
<br/>
For H8 and H16, find a preimage of the corresponding hashes: "\00" and "\00"\*2.
Measure how long it takes to find a preimage.


In [1]:
import hashlib
import time

# the target value of the question
h8_target = b"\00"
h16_target = b"\x00" * 2


def find_collision_h8(target):
    # attempting number
    attempt = 0
    # start counting execute time
    start_time = time.time()
    while True:
        # get the byte value of attempting time
        value_in_byte = str(attempt).encode()
        # hash the value of attempting time to byte then take the first value (take 1 byte = 8 bits)
        hash_value = hashlib.sha512(value_in_byte).digest()[:1]
        # if hash_value the same with target, stop the timer and break the loop
        if target == hash_value:
            end_time = time.time()
            print(
                f"Found collision of H8 after {attempt} attempt and {end_time - start_time:.6f} milliseconds"
            )
            break
        # increasing the attempting value if can not find the collision
        attempt += 1


def find_collision_h16(target):
    # attempting number
    attempt = 0
    # start counting execute time
    start_time = time.time()
    while True:
        # get the byte value of attempting time
        value_in_byte = str(attempt).encode()
        # hash the value of attempting time to byte then take the first value (take 1 byte = 8 bits)
        hash_value = hashlib.sha512(value_in_byte).digest()[:2]
        # if hash_value the same with target, stop the timer and break the loop
        if target == hash_value:
            end_time = time.time()
            print(
                f"Found collision of H16 after {attempt} attempt and {end_time - start_time:.6f} milliseconds"
            )
            break
        # increasing the attempting value if can not find the collision
        attempt += 1


find_collision_h8(h8_target)
find_collision_h16(h16_target)

Found collision of H8 after 61 attempt and 0.000997 milliseconds
Found collision of H16 after 288946 attempt and 0.377740 milliseconds


**Exercise 4**
<br/>
Find two messages that produce the same tag for AES-based CBC-MAC. Show code
to demonstrate that


In [4]:
!pip install pycryptodome


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

# In this answer we just using AES based CBC MAC, so the length will be a constant
BLOCK_SIZE = AES.block_size


# Encrypt message with CBC
def encrypt_aes_cbc(key, message):
    IV = bytes(BLOCK_SIZE)
    cipher = AES.new(key, AES.MODE_CBC, IV)
    return cipher.encrypt(message)


# Generate MAC tag for message with CBC
def generate_aes_cbc_mac_tag(key, message):
    cipher_text = encrypt_aes_cbc(key, message)
    # tag of MAC is the last {BLOCK_SIZE} bit of cipher text
    tag = cipher_text[-BLOCK_SIZE:]
    return tag


# XOR 2 bytes
def byte_xor(a, b):
    return bytes([_a ^ _b for _a, _b in zip(a, b)])


# Generate a random key with 16 bytes length
key = get_random_bytes(16)

# Generate message a with 32 bytes length
# a1: first block with 16 bytes length
# a2: first block with 16 bytes length
a1 = get_random_bytes(16)
a2 = get_random_bytes(16)
a = a1 + a2

# b1: first block with 16 bytes length of message b
b1 = get_random_bytes(16)

# Encrypt a1 with CBC
a1_encrypted = encrypt_aes_cbc(key, a1)
# Encrypt b1 with CBC
b1_encrypted = encrypt_aes_cbc(key, b1)

# XOR the a1_encrypted with b1_encrypted
xor_encryptions = byte_xor(a1_encrypted, b1_encrypted)

# Calculate b2 by XOR a2 with xor_encryptions
b2 = byte_xor(xor_encryptions, a2)

# concatenate b1 and b2 to get message b
b = b1 + b2
# show the result
print("Message a: ", a.hex())
print("MAC a:     ", generate_aes_cbc_mac_tag(key, a).hex())
print("Message b: ", b.hex())
print("MAC b:     ", generate_aes_cbc_mac_tag(key, b).hex())

Message a:  74ddee44f9d09dff5093055d6e8e6256e7b3905844ac223ca856f05e4467c20a
MAC a:      4f141fbd65a09cd7268d34ad5d4e7828
Message b:  9d708aaace1a0080cf80f98f71f486d375eaa8fb7178dd9fa40f2e0c05b97c9a
MAC b:      4f141fbd65a09cd7268d34ad5d4e7828
